In [1]:
import numpy as np
import pandas as pd

In [2]:
import re
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from nltk.stem import WordNetLemmatizer 

In [3]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\khush\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\khush\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\khush\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\khush\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [4]:
def preprocess_text(sen):
    # lower the character
    sentence = sen.lower()
    
    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)
    
    stops = stopwords.words('english')
    
    for word in sentence.split():
        if word in stops:
            sentence = sentence.replace(word, '')

    sentence = re.sub(r"http\S+", "", sentence)
    lemmatizer = WordNetLemmatizer()
    for word in sentence.split():
        sentence =sentence.replace(word, lemmatizer.lemmatize(word))
        
    return sentence

In [5]:
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


In [6]:
dg=pd.read_csv('troll.csv')

In [7]:
dg.head()

,Unnamed: 0,class,tweet
0,0,1,Get fucking real dude.
1,1,1,She is as dirty as they come and that crook ...
2,2,1,why did you fuck it up. I could do it all day...
3,3,1,Dude they dont finish enclosing the fucking s...
4,4,1,WTF are you talking about Men? No men thats n...


In [8]:
dg = dg.drop(columns=['Unnamed: 0'])

In [9]:
dg.head()

,class,tweet
0,1,Get fucking real dude.
1,1,She is as dirty as they come and that crook ...
2,1,why did you fuck it up. I could do it all day...
3,1,Dude they dont finish enclosing the fucking s...
4,1,WTF are you talking about Men? No men thats n...


In [10]:
dg['tweet'] = dg['tweet'].apply(lambda x : preprocess_text(x))

In [11]:
token = Tokenizer(28164)
token.fit_on_texts(dg['tweet'])
text2 = token.texts_to_sequences(dg['tweet'])
text2 = pad_sequences(text2,maxlen=250)

In [12]:
y = dg['class'].values

In [13]:
X_train, X_test, y_train, y_test = train_test_split(text2, y, test_size=0.2, random_state=1, stratify=y)

In [14]:
from sklearn.ensemble import RandomForestClassifier

#Create a Gaussian Classifier
clf=RandomForestClassifier(n_estimators=100)

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)

y_pred=clf.predict(X_test)

In [15]:
from sklearn.metrics import accuracy_score
from sklearn import metrics

In [16]:
print('accuracy for random forest: ',accuracy_score(y_test, y_pred))
print(metrics.classification_report(y_test, y_pred))

accuracy for random forest:  0.9460134966258436
              precision    recall  f1-score   support

           0       0.97      0.94      0.95      2436
           1       0.91      0.96      0.93      1565

    accuracy                           0.95      4001
   macro avg       0.94      0.95      0.94      4001
weighted avg       0.95      0.95      0.95      4001



In [17]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression(max_iter=10000)
LR.fit(X_train,y_train)
y_pred_LR = LR.predict(X_test)
print("Logistic Regression")
print("Accuracy score =", accuracy_score(y_test, y_pred_LR))
print(metrics.classification_report(y_test, y_pred_LR))

Logistic Regression
Accuracy score = 0.6075981004748813
              precision    recall  f1-score   support

           0       0.61      0.98      0.75      2436
           1       0.48      0.03      0.06      1565

    accuracy                           0.61      4001
   macro avg       0.54      0.50      0.41      4001
weighted avg       0.56      0.61      0.48      4001



In [18]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors = 5)
neigh.fit(X_train,y_train)
y_pred_KNN = neigh.predict(X_test)
print("KNN")
print("Accuracy score =", accuracy_score(y_test, y_pred_KNN))
print(metrics.classification_report(y_test, y_pred_KNN ))

KNN
Accuracy score = 0.7980504873781554
              precision    recall  f1-score   support

           0       0.85      0.81      0.83      2436
           1       0.73      0.77      0.75      1565

    accuracy                           0.80      4001
   macro avg       0.79      0.79      0.79      4001
weighted avg       0.80      0.80      0.80      4001



In [19]:
from sklearn.naive_bayes import GaussianNB
naive = GaussianNB()
naive.fit(X_train,y_train)
y_pred_naive = naive.predict(X_test)
print("Naive Bayes")
print("Accuracy score =", accuracy_score(y_test, y_pred_naive))
print(metrics.classification_report(y_test, y_pred_naive ))

Naive Bayes
Accuracy score = 0.3976505873531617
              precision    recall  f1-score   support

           0       0.91      0.01      0.02      2436
           1       0.39      1.00      0.56      1565

    accuracy                           0.40      4001
   macro avg       0.65      0.50      0.29      4001
weighted avg       0.71      0.40      0.24      4001



In [20]:
from sklearn.ensemble import GradientBoostingClassifier
gradient = GradientBoostingClassifier(n_estimators=100,max_depth=None,min_samples_split=2, random_state=0)
gradient.fit(X_train,y_train)
y_pred_gradient = gradient.predict(X_test)
print("Gradient Boosting")
print("Accuracy score =", accuracy_score(y_test, y_pred_gradient))
print(metrics.classification_report(y_test, y_pred_gradient ))

Gradient Boosting
Accuracy score = 0.9385153711572107
              precision    recall  f1-score   support

           0       0.98      0.92      0.95      2436
           1       0.88      0.97      0.93      1565

    accuracy                           0.94      4001
   macro avg       0.93      0.94      0.94      4001
weighted avg       0.94      0.94      0.94      4001



In [21]:
from sklearn.tree import DecisionTreeClassifier
decision = DecisionTreeClassifier()
decision.fit(X_train,y_train)
y_pred_decision = decision.predict(X_test)
print("Decision Tree")
print("Accuracy score =", accuracy_score(y_test, y_pred_decision))
print(metrics.classification_report(y_test, y_pred_decision ))

Decision Tree
Accuracy score = 0.9325168707823044
              precision    recall  f1-score   support

           0       0.98      0.91      0.94      2436
           1       0.87      0.97      0.92      1565

    accuracy                           0.93      4001
   macro avg       0.93      0.94      0.93      4001
weighted avg       0.94      0.93      0.93      4001



In [ ]:
from hmmlearn import hmm


In [14]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(len(token.word_index)+1, 255, input_length=250),
    tf.keras.layers.Conv1D(32, 4),
    tf.keras.layers.Conv1D(16, 8),
    tf.keras.layers.Conv1D(8, 16),
    tf.keras.layers.Conv1D(4, 32),
    tf.keras.layers.Conv1D(2, 64),
    tf.keras.layers.Conv1D(1, 128),
    tf.keras.layers.Bidirectional(tf.keras.layers.GRU(100, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.GRU(100, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.GRU(100)),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(512, 'relu'),
    tf.keras.layers.Dense(1, 'sigmoid')
])

adam = tf.keras.optimizers.Adam()
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 250, 255)          4476015   
_________________________________________________________________
conv1d (Conv1D)              (None, 247, 32)           32672     
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 240, 16)           4112      
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 225, 8)            2056      
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 194, 4)            1028      
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 131, 2)            514       
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 4, 1)              2

In [15]:
model.fit(text2 , y , epochs=3, verbose=1, batch_size=64)

Epoch 1/3
313/313 [==============================] - 100s 224ms/step - loss: 0.6727 - accuracy: 0.6048
Epoch 2/3
313/313 [==============================] - 71s 226ms/step - loss: 0.6635 - accuracy: 0.6165
Epoch 3/3
313/313 [==============================] - 69s 222ms/step - loss: 0.3515 - accuracy: 0.8502


In [45]:
X = 'Lick my balls'

In [27]:
import pickle

# saving
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(token, handle, protocol=pickle.HIGHEST_PROTOCOL)

# loading
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [28]:
import pickle
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [29]:
tokenizer.texts_to_sequences(['hat'])

[[801]]

In [38]:
import joblib
joblib.dump(clf, "random_forest.joblib")

['random_forest.joblib']

In [36]:
import joblib
loaded_rf = joblib.load("random_forest.joblib")

In [41]:
text4 = tokenizer.texts_to_sequences(['hate'])
padzero(text4[0])

In [42]:
text4

[[0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,


In [43]:
loaded_rf.predict(text4)

array([1], dtype=int64)

In [44]:
text4 = token.texts_to_sequences(['Not funny'])

In [45]:
text4

[[167]]

In [46]:
text2[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   

In [34]:
def padzero(lst):
    for i in range(250-len(lst)):
        lst.insert(0,0)

In [48]:
padzero(text4[0])

In [49]:
text4

[[0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,


In [50]:
loaded_rf.predict(text4)

array([0], dtype=int64)

In [17]:
def Predict(seed_text):
    seed_text = token.texts_to_sequences(seed_text)
    seed_text = tf.keras.preprocessing.sequence.pad_sequences(seed_text, 250)
    prediction = model(seed_text)
    print(str(prediction))

In [23]:
Predict(["The world is being controlled by one crazy criminal"])

tf.Tensor([[0.00823036]], shape=(1, 1), dtype=float32)
